# PAML site test submit sbatch job

In [ ]:
# Import all necessary packages
import os
import pandas as pd
import subprocess
from pathlib import Path
import random, shutil, subprocess, re
from io import StringIO
from typing import List, Tuple
from Bio import AlignIO, SeqIO, Phylo
from Bio.Align import MultipleSeqAlignment
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import shutil

## Workflow 

In [ ]:
# How the output files will look: 
# {data_dir}/PAML/x_chromosome/{family}/{cluster}/
#   ├─ {cluster}_nogaps.fa         (once per cluster)
#   ├─ iter_01/
#   │    ├─ subset_species.fasta
#   │    ├─ subset.phy
#   │    ├─ subset_fullID_to_species.tsv
#   │    ├─ tree_pruned.nwk
#   │    ├─ codeml.ctl
#   │    └─ mlc.txt                (codeml output)
#   ├─ iter_02/  (same files)
#   └─ iter_03/  (same files)

In [ ]:
# Extract clusters 
# set the family names for the X chromosome
# families = ['CSF2RA', 'SPANX', 'TBL1X' ,'VCX' ,'TMSB' ,'MAGEB',
#  'TCEAL8' ,'H2A','endogenous','SPACA5',
#  'SSX' ,'GAGE' ,'NUDT10' ,'CENPVL',
#  'FLJ39060' ,'XAGE1' ,'FAM156', 'SPIN',
#  'ZXD' ,'CXorf49' ,'DMRTC1' ,'FAM236', 'PABPC', 'RPL36A', 'ARMCX' ,'NXF',
#  'TCP11X2' ,'GPRASP', 'RAB40A' ,'H2BW', 'CT47' ,'RHOXF2' ,'SMIM10' ,'ETD',
#  'INTS6L', 'CT45A', 'CXorf51', 'EOLA' ,'HSFX' ,'TMEM185A', 'CSAG', 'PNMA',
#  'PWWP4', 'OPN1LW', 'TEX28', 'LAGE3', 'IKBKG' ,'F8A1',
#  'collagen' ,'LOC129475109','LOC115932372']

# set the family names for the Y chromosome
families = ['CDY1', 'glutamate', 'TSPY8' ,'DAZ1',
 'BPY2', 'RBMY1B', 'MTRNR2', 'proline', 'VCY1B',
 'HSFY1', 'keratin' ,'FRG1',
 'centriole','FAM47A', 'zinc','isoenzyme',
 'retrovirus','TATA-box']


In [ ]:

data_dir = Path("/home/emma/Amplicons/Workspaces/emma/downloaded_data")

# species codes + fixed species tree (all taxa; Will be pruned per iteration)
species_targets = ["HomSap","PanTro","PanPan","GorGor","PonAbe","PonPyg","SymSyn","MacFas"]
# Unrooted tree
tree_newick = "(((((PanTro,PanPan), HomSap), GorGor), (PonPyg,PonAbe)), SymSyn, MacFas);"

# tools
perl_fasta2phy = data_dir / "PAML" / "FASTAtoPHYL.pl"   # your Perl converter
codeml_bin = "codeml"                                    # or Path("/full/path/to/codeml")

# iterations
n_iters = 6 
base_seed = None 

# base output dir
PAML_ROOT = data_dir / "PAML" / "x_chromosome" # for the X chromosome
#PAML_ROOT = data_dir / "PAML" / "y_chromosome" # for the y chromosome

PAML_ROOT.mkdir(parents=True, exist_ok=True)

In [ ]:
# extract the clusters per family
# this dict will hold, for each family, the list of multi-seq clusters
# change the directory sequences_x to sequences_y when doing one or the other. 

cluster_list_per_family = {}

for family in families:
    # ensure the alignments directory exists
    cluster_alignments = f"{data_dir}/sequences_y/{family}_selected_isoform/blastdb/cluster_alignments"
    os.makedirs(cluster_alignments, exist_ok=True)

    # grab every .fa basename in the cluster_fastas dir
    cluster_dir = f"{data_dir}/sequences_y/{family}_selected_isoform/blastdb/cluster_fastas"
    all_clusters = [
        os.path.splitext(fn)[0]
        for fn in os.listdir(cluster_dir)
        if fn.endswith(".fa")
    ]

    # filter out FASTAs with only one sequence
    filtered = []
    for name in all_clusters:
        path = os.path.join(cluster_dir, f"{name}.fa")
        with open(path) as f:
            nseq = sum(1 for line in f if line.startswith(">"))
        if nseq > 1:
            filtered.append(name)

    # check for duplicate IDs
    for name in filtered:
        path = os.path.join(cluster_dir, f"{name}.fa")
        seen, dups = set(), set()
        with open(path) as f:
            for line in f:
                if line.startswith(">"):
                    seqid = line[1:].split()[0]
                    if seqid in seen:
                        dups.add(seqid)
                    else:
                        seen.add(seqid)
        if dups:
            print(f"[{family}] {name}.fa has duplicate IDs: {', '.join(dups)}")

    # store the filtered list for later
    cluster_list_per_family[family] = filtered

    print(f"{family}: keeping {len(filtered)} clusters")


SSX: keeping 3 clusters


In [ ]:
# All functions
# Clean up the alignment -> codeml has a hard time with gaps. So drop entire codons if any sequence has a gap inside that codon
def make_codonwise_nogap_alignment(inp_path, out_path, gap_chars="-"):
    from Bio import AlignIO
    from Bio.Align import MultipleSeqAlignment
    from Bio.SeqRecord import SeqRecord
    from Bio.Seq import Seq

    aln = AlignIO.read(str(inp_path), "fasta")
    L = aln.get_alignment_length()
    floorL = (L // 3) * 3

    keep_idx = []
    gapset = set(gap_chars)
    for start in range(0, floorL, 3):
        # drop codon if ANY '-' occurs in its 3 columns (in ANY sequence)
        if any(any(ch in gapset for ch in aln[:, start + k]) for k in range(3)):
            continue
        keep_idx.extend([start, start+1, start+2])

    if not keep_idx:
        raise ValueError("No codon columns left after codon-wise gap removal.")

    trimmed = MultipleSeqAlignment([
        SeqRecord(Seq(''.join(str(rec.seq)[i] for i in keep_idx)), id=rec.id, description="")
        for rec in aln
    ])

    if trimmed.get_alignment_length() % 3 != 0:
        raise AssertionError("Trimmed length not divisible by 3 (unexpected).")

    AlignIO.write(trimmed, str(out_path), "fasta")
    kept_codons = len(keep_idx) // 3
    total_codons = floorL // 3
    print(f"codonwise nogaps -> {out_path.name} (kept {kept_codons}/{total_codons} codons)")

def pick_one_per_species(fasta_path: Path, targets: List[str], rng: random.Random) -> Tuple[List[SeqRecord], dict, int, int]:
    records = list(SeqIO.parse(str(fasta_path), "fasta"))
    buckets = {sp: [] for sp in targets}
    for r in records:
        sp = r.id.split("_")[-1]
        if sp in buckets:
            r.seq = r.seq.upper()
            buckets[sp].append(r)
    picked, id_map = [], {}
    for sp in targets:
        if buckets[sp]:
            r = rng.choice(buckets[sp])
            picked.append(r)
            id_map[r.id] = sp
    if len(picked) < 2:
        raise ValueError("Need at least 2 target species in this cluster.")
    lens = {len(r.seq) for r in picked}
    if len(lens) != 1:
        raise ValueError(f"Unequal lengths after nogaps: {sorted(lens)}")
    return picked, id_map, len(picked), next(iter(lens))

def write_species_fasta(picked: List[SeqRecord], out_fa: Path):
    with open(out_fa, "w") as out:
        for r in picked:
            sp = r.id.split("_")[-1]
            out.write(f">{sp}\n{str(r.seq)}\n")

def prune_tree_to_species(tree_newick: str, present: List[str], out_tree: Path):
    tree = Phylo.read(StringIO(tree_newick), "newick")
    names = {t.name for t in tree.get_terminals()}
    missing = sorted(set(present) - names)
    if missing:
        raise ValueError(f"Tree missing taxa present in alignment: {missing}")
    for term in list(tree.get_terminals()):
        if term.name not in present:
            tree.prune(term)
    Phylo.write(tree, str(out_tree), "newick")


def fasta_to_phylip_with_perl(perl_script: Path, species_fa: Path, nseq: int, length: int, out_phy: Path):
    # run in the iteration folder so the .phy is created there
    cwd = species_fa.parent
    cmd = ["perl", str(perl_script), species_fa.name, str(nseq), str(length)]
    subprocess.run(cmd, check=True, cwd=str(cwd))
    tmp = cwd / (species_fa.stem + ".phy")
    if not tmp.exists():
        raise FileNotFoundError(f"Expected {tmp} not found after Perl conversion.")
    shutil.move(str(tmp), str(out_phy))

def write_ctl(ctl_path: Path, seqfile: str, treefile: str, outfile: str):
    ctl = f"""seqfile = {seqfile}
treefile = {treefile}
outfile = {outfile}

noisy = 3                              * Display moderate amount of information on the screen
verbose = 1                            * Detailed output file
seqtype = 1                            * Codon data
ndata = 1                              * One gene alignment
icode = 0                              * Universal genetic code
cleandata = 0                          * Do not remove sites with ambiguity data (because gaps already removed before)
model = 0                              * One ω for all branches (M0 and site models)
NSsites = 0 1 2 7 8                    * Models M0 (0), M1a (1), M2a (2), M7 (7), and M8 (8)
CodonFreq = 7                          * Use mutation-selection model
estFreq = 0                            * Use observed frequencies to calculate fitness/freq pars
clock = 0                              * Assume no clock
fix_omega = 0                          * Enables option to estimate omega
omega = 0.5                            * Initial omega value

"""
    ctl_path.write_text(ctl)

def run_codeml(codeml_bin, ctl_path: Path, workdir: Path):
    subprocess.run([str(codeml_bin), ctl_path.name], cwd=str(workdir), check=True)

In [ ]:
# run the Workflow

if shutil.which("perl") is None:
    raise SystemExit("Perl not found. Install perl or add it to PATH.")
if shutil.which(str(codeml_bin)) is None:
    raise SystemExit("codeml not found. Set 'codeml_bin' to its full path or add to PATH.")
if not perl_fasta2phy.exists():
    raise SystemExit(f"Perl converter not found: {perl_fasta2phy}")

for family in families:
    clusters = cluster_list_per_family.get(family, [])
    print(f"\n=== FAMILY: {family} ({len(clusters)} clusters) ===")
    for cluster in clusters:
        # input alignment to clean (your trimmed alignments live here) # SEQUENCES_X HAS TO BE CHANGED TO SEQUENCES_Y WHEN Y CHROM
        inp = data_dir / "sequences_y_longestisoform" / f"{family}_selected_isoform" / "blastdb" / "cluster_alignments" / f"{cluster}_NT.fa"
        if not inp.exists():
            print(f"  [skip] missing alignment: {inp}")
            continue

        cluster_dir = PAML_ROOT / family / cluster
        cluster_dir.mkdir(parents=True, exist_ok=True)

        # 1) no-gap alignment once per cluster
        nogap = cluster_dir / f"{cluster}_nogaps.fa"
        if not nogap.exists():
            make_codonwise_nogap_alignment(inp, nogap)

        # RNG for this cluster
        rng = random.Random(base_seed) if base_seed is not None else random.Random()

        # 2) iterations
        for it in range(1, n_iters+1):
            iter_dir = cluster_dir / f"iter_{it:02d}"
            iter_dir.mkdir(exist_ok=True)

            try:
                # pick one seq per species
                picked, id_map, nseq, length = pick_one_per_species(nogap, species_targets, rng)
                present = sorted({r.id.split("_")[-1] for r in picked})

                # write subset files
                subset_fa   = iter_dir / "subset_species.fasta"
                subset_phy  = iter_dir / "subset.phy"
                subset_map  = iter_dir / "subset_fullID_to_species.tsv"
                subset_tree = iter_dir / "tree_pruned.nwk"

                write_species_fasta(picked, subset_fa)
                with open(subset_map, "w") as f:
                    f.write("full_id\tspecies\n")
                    for full, sp in id_map.items():
                        f.write(f"{full}\t{sp}\n")

                prune_tree_to_species(tree_newick, present, subset_tree)

                # FASTA -> PHYLIP
                fasta_to_phylip_with_perl(perl_fasta2phy, subset_fa, nseq, length, subset_phy)

                # codeml control + run
                ctl = iter_dir / "codeml.ctl"
                out_mlc = iter_dir / "mlc.txt"
                write_ctl(ctl, "subset.phy", "tree_pruned.nwk", "mlc.txt")
                run_codeml(codeml_bin, ctl, iter_dir)

                print(f"  {family}/{cluster} iter {it:02d}: OK ({nseq} spp × {length} bp)")

            except Exception as e:
                print(f"  {family}/{cluster} iter {it:02d}: SKIP -> {e}")



=== FAMILY: SSX (3 clusters) ===
Parsing alignment subset_species ... ... 
Total no. species parsed: 2
Last seq length: 726

CODONML in paml version 4.10.9, 7 May 2025

----------------------------------------------
Phe F TTT | Ser S TCT | Tyr Y TAT | Cys C TGT
      TTC |       TCC |       TAC |       TGC
Leu L TTA |       TCA | *** * TAA | *** * TGA
      TTG |       TCG |       TAG | Trp W TGG
----------------------------------------------
Leu L CTT | Pro P CCT | His H CAT | Arg R CGT
      CTC |       CCC |       CAC |       CGC
      CTA |       CCA | Gln Q CAA |       CGA
      CTG |       CCG |       CAG |       CGG
----------------------------------------------
Ile I ATT | Thr T ACT | Asn N AAT | Ser S AGT
      ATC |       ACC |       AAC |       AGC
      ATA |       ACA | Lys K AAA | Arg R AGA
Met M ATG |       ACG |       AAG |       AGG
----------------------------------------------
Val V GTT | Ala A GCT | Asp D GAT | Gly G GGT
      GTC |       GCC |       GAC |       GG


error: end of tree file.

 0.0010 -------N   938.644419  0 0.0000   533
Out..
lnL  =  -938.644419
534 lfun, 5874 eigenQcodon, 10680 P(t)


NSsites Model 8: beta&w>1

TREE #  1
(2, 1);   MP score: -1
    0.013293    0.003384    3.771549    0.022946    0.006408    0.000100    0.900000    0.871667    1.344001  999.000000

ntime & nrate & np:     2     5    10

Bounds (np=10):
   0.000004   0.000004   0.000100   0.000100   0.000100   0.000100   0.000010   0.005000   0.005000   1.000000
  50.000000  50.000000 999.000000 999.000000 999.000000 999.000000   0.999990  99.000000  99.000000 999.000000
Qfactor_NS = 0.071385

np =    10
lnL0 =  -938.303608

Iterating by ming2
Initial: fx=   938.303608
x=  0.01329  0.00338  3.77155  0.02295  0.00641  0.00011  0.90000  0.87167  1.34400 951.42857

  1 h-m-p  0.0000 0.0000 1098.4256 ++      938.298087  m 0.0000    15 | 1/10
  2 h-m-p  0.0000 0.0013 1350.4676 -CCC    938.294658  2 0.0000    33 | 1/10
  3 h-m-p  0.0001 0.0031   9.7368 +CCC    938.283055  2 0.0004    51 | 1/10
  4


error: end of tree file.

 939.260097  5 0.0000    56 | 2/10
  3 h-m-p  0.0002 0.0008  51.5708 +YYCYCCC   938.425981  6 0.0005    87 | 2/10
  4 h-m-p  0.0023 0.0116   3.7769 YYYY    938.408984  3 0.0022   111 | 1/10
  5 h-m-p  0.0003 0.0016  22.7109 CC      938.405624  1 0.0001   134 | 1/10
  6 h-m-p  0.0208 0.2650   0.1375 ++      938.378151  m 0.2650   156 | 2/10
  7 h-m-p  0.0795 8.0000   0.4584 +CCC    938.337909  2 0.2984   183 | 2/10
  8 h-m-p  0.3025 2.1561   0.4521 YCCC    938.295151  3 0.5510   209 | 2/10
  9 h-m-p  0.5649 6.1157   0.4410 YCCC    938.234007  3 1.3736   235 | 2/10
 10 h-m-p  0.8383 8.0000   0.7226 +YCCC   938.042780  3 5.3534   262 | 2/10
 11 h-m-p  1.6000 8.0000   0.2888 YCC     938.028411  2 0.9765   286 | 2/10
 12 h-m-p  1.6000 8.0000   0.0349 +CC     938.027008  1 5.4940   310 | 2/10
 13 h-m-p  1.6000 8.0000   0.0097 ---------------N   938.027008  0 0.0000   346 | 2/10
 14 h-m-p  0.0160 8.0000   0.4127 ++YC    938.026332  1 0.4596   370 | 2/10
 15 h-m-p  1.6000 8.0000   0.0849 YC   

...


error: end of tree file.

0 h-m-p  1.6000 8.0000   0.1495 CC     1502.094589  1 0.3518  1509 | 1/21
 41 h-m-p  0.1895 3.4123   0.2775 YC     1502.093430  1 0.3802  1554 | 1/21
 42 h-m-p  0.4576 4.1088   0.2306 CYC    1502.092520  2 0.8406  1601 | 1/21
 43 h-m-p  1.6000 8.0000   0.0709 YC     1502.091432  1 0.7330  1646 | 1/21
 44 h-m-p  0.7926 8.0000   0.0656 C      1502.091325  0 0.7926  1690 | 1/21
 45 h-m-p  1.6000 8.0000   0.0043 C      1502.091281  0 1.5323  1734 | 1/21
 46 h-m-p  0.6963 8.0000   0.0095 +Y     1502.091263  0 2.1088  1779 | 1/21
 47 h-m-p  1.6000 8.0000   0.0049 Y      1502.091262  0 1.2116  1823 | 1/21
 48 h-m-p  1.6000 8.0000   0.0009 Y      1502.091261  0 1.0419  1867 | 1/21
 49 h-m-p  1.6000 8.0000   0.0003 Y      1502.091261  0 1.1291  1911 | 1/21
 50 h-m-p  1.6000 8.0000   0.0000 C      1502.091261  0 1.5067  1955 | 1/21
 51 h-m-p  1.6000 8.0000   0.0000 C      1502.091261  0 0.5028  1999 | 1/21
 52 h-m-p  0.2639 8.0000   0.0000 C      1502.091261  0 0.2639  2043 | 1/21
 53 h-m-p  0.3


error: end of tree file.

061  2519 | 4/21
 57 h-m-p  0.0071 2.5342   2.3845 Y      1688.240963  0 0.0030  2560 | 4/21
 58 h-m-p  0.0098 0.7654   0.7211 C      1688.240942  0 0.0022  2601 | 4/21
 59 h-m-p  0.0120 6.0105   0.2518 +C     1688.240800  0 0.0436  2643 | 4/21
 60 h-m-p  0.0061 3.0609   6.6695 YC     1688.239765  1 0.0121  2685 | 4/21
 61 h-m-p  0.0917 8.0000   0.8791 ++YC   1688.229751  1 0.9189  2729 | 4/21
 62 h-m-p  0.6737 8.0000   1.1991 YC     1688.213116  1 1.0926  2771 | 4/21
 63 h-m-p  0.7747 8.0000   1.6911 +YCC   1688.159779  2 2.4503  2816 | 4/21
 64 h-m-p  1.6000 8.0000   0.1940 CC     1688.155868  1 0.4991  2859 | 4/21
 65 h-m-p  0.4560 8.0000   0.2124 CC     1688.155210  1 0.6515  2902 | 4/21
 66 h-m-p  1.6000 8.0000   0.0485 Y      1688.155198  0 0.9558  2943 | 4/21
 67 h-m-p  1.6000 8.0000   0.0022 Y      1688.155198  0 0.8866  2984 | 4/21
 68 h-m-p  1.6000 8.0000   0.0001 Y      1688.155198  0 0.8483  3025 | 4/21
 69 h-m-p  1.6000 8.0000   0.0000 --------C  1688.155198  0 0.0000  307


error: end of tree file.